# 텍스트 생성 하기

## 1. GPU 설정

In [2]:
# 머신러닝 모델 GPU 사용하기 - torch
## 1. GPU 사용 여부 확인
import torch

## GPU 사용 가능 여부 확인
print(torch.cuda.is_available())        # True라면 GPU 사용 가능
print(torch.cuda.device_count())        # 사용 가능한 GPU 개수 출력
print(torch.cuda.get_device_name(0))    # 첫 번째 GPU 이름 출력

# 2. 디바이스 설정 (GPU가 없으면 CPU 사용)
device = torch.device(1)
print(f"Using device: {device}")

True
2
NVIDIA GeForce RTX 4090
Using device: cuda:1


## 2. 텍스트 모델로 가사 생성하기

### 1) 텍스트 모델 개발 VER-01

In [18]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, PreTrainedTokenizerFast, GPTNeoForCausalLM, GPTNeoForCausalLM
from transformers import AutoTokenizer, AutoModelForCausalLM
from tqdm import tqdm
import torch

# 1) 모델 불러오기
model_name = "skt/kogpt2-base-v2"
# KoGPT2 : "skt/kogpt2-base-v2"
# Llama-2 (한국어 지원 포함) : "meta-llama/Llama-2-7b-chat-hf" 
# kobart (한국어 요약 및 번역) : "gogamza/kobart-base-v2"
# EleutherAI/polyglot-ko-1.3b (한글 전용 GPT 모델) : "EleutherAI/polyglot-ko-1.3b"          # 한글 텍스트 생성에 특화
# EleutherAI/gpt-neo-2.7B (멀티언어 모델) : "EleutherAI/gpt-neo-2.7B"
# beomi/KoGPT-6B (GPT 계열 대형 모델) : "beomi/KoGPT-6B"                                    # 한글 텍스트 생성에 특화


tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)             # Tokenizer 자동 매치
# tokenizer = GPT2Tokenizer.from_pretrained(model_name)           # Tokenizer GPT2 기반 매치 : KoGPT2가 호환이 되는지 잘 모르겠음

# 2) 모델 설정하기
# model = AutoModelWithLMHead.from_pretrained(model_name)         # AutoModelWithLMHead 향후 버전에서 제거될 예정
# model = AutoModelForSeq2SeqLM.from_pretrained(model_name)       # AutoModelForSeq2SeqLM : Sequence-to-Sequence Language Model (Seq2Seq), 예를 들어 T5, BART처럼 텍스트 변환 작업에 사용
# model = AutoModelForMaskedLM.from_pretrained(model_name)        # AutoModelForMaskedLM : Masked Language Model (MLM), 예를 들어 BERT 계열 모델처럼 텍스트 마스킹과 복원을 위해 사용
model = GPT2LMHeadModel.from_pretrained(model_name)               # AutoModelForCausalLM : Causal Language Model (CLM), 예를 들어 GPT 계열 모델처럼 텍스트 생성에 사용

# 3) 데이터 불러오기
import pandas as pd
table_data = pd.read_csv(r"/home/wanted-1/potenup-workspace/Project/project2/team5/1.데이터모음/'music_data(Merge)'.csv")
table_data["lyrics"].astype(str)
input_text = "\n".join(table_data["lyrics"].dropna())  

# 4) 토큰화
def token_input(text):
    result = []
    for string in tqdm(text, desc="Tokenize Processing : ") :
        result.extend(tokenizer.encode(string, return_tensors="pt", truncation=True, max_length=50))
        # return_tensors : 토큰화된 결과를 특정 텐서 형식으로 반환하도록 설정하는 옵션, 프레임워크에 맞게 데이터 타입을 조정하는 데 사용
            # "pt" : Pytorch 텐서
            # "tf" : TensorFlow 텐서
            # "np" : Numpy 배열
            # None : 기본값, 반환 타입은 Python list
    return result
inputs = token_input(input_text)
inputs_cat = torch.cat(inputs, dim=0)
max_input_length = 1024
input_ids = inputs_cat[:max_input_length]

# 5) 텍스트 생성
batch_size = 16
outputs = []
for i in tqdm(range(0, input_ids.size(0), batch_size), desc="Batch Processing : "):
    batch = input_ids[i:i + batch_size]                                                     # i부터 batch_size만큼 슬라이싱
    if batch.size(0) > 1024:                                                                # 최대 입력 길이를 제한
        batch = batch[:1024]                                                                # 슬라이싱
    if batch.dim() == 1:                                                                    # 1차원일 경우, 2차원으로 변경 필요
        batch = batch.unsqueeze(0)                                                          # 배치 차원 추가
    output = model.generate(batch, max_new_tokens=50, do_sample=True)
    outputs.append(output)
    
# outputs의 구조 확인
print(type(outputs))  # 리스트인지 확인
print(type(outputs[0]))  # 첫 번째 요소의 타입 (예: torch.Tensor)
print(outputs[0].shape)  # 첫 번째 요소의 크기
print(outputs[0])

Batch Processing : 100%|██████████| 64/64 [00:35<00:00,  1.79it/s]

<class 'list'>
<class 'torch.Tensor'>
torch.Size([1, 66])
tensor([[11045, 16901, 13008, 11925, 10288, 11849, 16901,   739, 10448, 10288,
           739, 11849, 19490,   739, 18896, 10288, 11849, 10448,   739,   739,
           459,   387,   375,   463, 19490, 11849, 10448,   739,   459,   387,
           375,   463, 11780, 22272, 10288, 11849, 10448, 11481,   375,   463,
         18125, 22272,   739,   459,   387,   375,   463, 30266, 18125, 22272,
           375,   461, 18125, 22272,   387,   375,   463, 11780, 22272,   387,
           375,   457, 22272,   387,   375,   463]])


In [19]:
# 6) 결과 출력
import random

def token_output(tensor_outputs):
    result = []
    for tensor in tqdm(tensor_outputs, desc="Decoding Processing : "):
        decoded = tokenizer.decode(tensor[0].tolist(), skip_special_tokens=True)
        result.append(decoded)
    return result

def clean_text(generated_texts):
    cleaned_texts = []
    for text in generated_texts:
        cleaned_text = text.replace(" ", "")
        cleaned_texts.append(cleaned_text)
    return cleaned_texts


generated_text = token_output(outputs)
cleaned_text = clean_text(generated_text)
print("\n".join(cleaned_text))

Decoding Processing : 100%|██████████| 64/64 [00:00<00:00, 7640.12it/s]

Welcometomywomtu,
yymtu,
yskomt,
yhku,
ylehk
whk,
ysk,
sk,
y
rld

Red,Blue

H
Duuu+a
Huuuuuuuxx
Eeu~
Huuxxuxx

,Green

Mywor
M
Xxyyy
S×yx
XyyX
Oyy
M
Oyyy
ldisfullofb.lciseuulliseuuuuleuuutwmuu.
fu는isu
lue

There'snoialoi
Ialoiaga
Ialoialoi
Ialoialoi
Ialoialoi
Ialoi
lightinyourra
n에iso
w이rra
wr이ra
lpra
wra
ln
clra
inidra
wra
lpl
world

Red,Bl.
E
Red
Red
Red
Red
Red
Red
Red
Red
Red
Red
Red
Re
ue,Green

No

AE
DEEB
Gfee?
EEM
FIM
Gfe
JM
DEE
FIM
EM
EM
EM
JM

Iwillabsorbyorglnabtxn
이런곡이었어요.
자그러면첫곡까지모두다세번째곡을가져갈수있어야하죠?
오늘의마지막곡세번째곡이에요.
두번째곡이에요.
이번곡도
urbluelight,rup4xeup5up5uedeuf
uruedeup6u7u3u1u(p)
u
likeredlight
egiaelgh!
elpqreerenreere
ekeeedengt
lok
이

IfellintotheelimIheeilhoqlhoqlhtelheeilimI\eil\rightthrtehrtehr
deepsea

Ifeeape)
O(e)
R(e)
T(e)
Terror
B(e)코우가는코우가를만나기전시합을위해떠나기전코우가와
llikeI'mlost
'mssfevthyisagoodnotearetosay
'''howtothebestbestgirl.aretoosaylice
''

Youwantabrisrit
picqqqq
qqqqqq
이번주에는BR과KR,그리고IBM과같은유수의글로벌서버
ghtlight

Iknotc
wwhatyouwantumatatambguanksaksbkgatsaksnkkksnks

### 2) 텍스트 모델 개발 VER-02

### 3) 텍스트 모델 개발 VER-03

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from torch.utils.data import DataLoader, Dataset
from transformers import AdamW
from tqdm import tqdm
from torch import torch 
from transformers import get_scheduler

# KoGPT2 토크나이저 불러오기
tokenizer = AutoTokenizer.from_pretrained("skt/kogpt2-base-v2")

# pad_token 설정 
tokenizer.pad_token = tokenizer.eos_token

# 데이터 읽기
with open("lyrics.txt", "r", encoding="utf-8") as file:
    raw_text = file.read()

# 토큰화
tokens = tokenizer(raw_text, return_tensors="pt", max_length=1024, truncation=True, padding=True)

# 모델 로드
model = AutoModelForCausalLM.from_pretrained("skt/kogpt2-base-v2")

# 토큰 추가가 있다면 모델 업데이트
model.resize_token_embeddings(len(tokenizer))


#  데이터셋 클래스 정의
class LyricsDataset(Dataset):
    def __init__(self, tokens):
        self.tokens = tokens["input_ids"]
        self.attention_mask = tokens["attention_mask"]
    
    def __len__(self):
        return len(self.tokens)
    
    def __getitem__(self, idx):
        return {
            "input_ids": self.tokens[idx],
            "attention_mask": self.attention_mask[idx]
        }

#  데이터 로더 생성
batch_size = 8  # 배치 크기를 8로 증가
dataset = LyricsDataset(tokens)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


# 옵티마이저 및 학습률 스케줄러 설정
learning_rate = 5e-5  # 데이터셋에 적합한 학습률
weight_decay = 0.01   # 과적합 방지용 Weight Decay 설정
optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

# Warmup 단계 설정
epochs = 5  # 원하는 에포크 수
total_steps = len(dataloader) * epochs
warmup_steps = int(0.1 * total_steps)  # 전체 단계의 10%를 Warmup으로 설정
lr_scheduler = get_scheduler(
    "linear", optimizer=optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_steps
)

# 4. 학습 루프
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.train()

for epoch in range(epochs):
    epoch_loss = 0
    for batch in tqdm(dataloader, desc=f"Training Epoch {epoch+1}"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)

        # 모델 출력 계산
        outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)
        loss = outputs.loss
        epoch_loss += loss.item()

        # 역전파 및 가중치 업데이트
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # 학습률 스케줄러 업데이트
        lr_scheduler.step()

    #print(f"Epoch {epoch+1} Loss: {epoch_loss:.4f}")

# 학습 모델 저장 
model.save_pretrained("./kogpt2-lyrics")
tokenizer.save_pretrained("./kogpt2-lyrics")

# 학습 된 모델 및 토크나이저 불러오기
tokenizer = AutoTokenizer.from_pretrained("./kogpt2-lyrics")
model.to(device)

# 초기 입력(prompt) 설정
prompt = "외로워"  # 가사 생성 시작 문구
input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)

# 생성 하이퍼파라미터 설정
generated = model.generate(
    input_ids=input_ids,
    max_length=200,  # 생성할 최대 길이
    temperature=1.0,  # 다양성 조절
    top_k=50,  # 상위 50개 후보 중에서 선택
    top_p=0.9,  # 누적 확률 기반 선택
    repetition_penalty=1.2,  # 반복 억제
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id=tokenizer.eos_token_id
)

#  결과 출력
generated_text = tokenizer.decode(generated[0], skip_special_tokens=True)
print(generated_text)

### 4) 텍스트 모델 개발 VER-04

In [5]:
from transformers import GPT2LMHeadModel, AutoTokenizer

# 저장된 모델 및 토크나이저 로드
model = GPT2LMHeadModel.from_pretrained("./fine_tuned_model2")
tokenizer = AutoTokenizer.from_pretrained("./fine_tuned_model2")

# 모델을 GPU로 이동
model.to("cuda")
model.eval()  # 평가 모드로 전환 (dropout 비활성화)

# 프롬프트 정의
prompt = "퇴근 후 나른한 오후에 느끼는 상반된 나의 감정, 쓸쓸함과 안락함"

# 텍스트를 토큰화
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")

# 텍스트 생성
with torch.no_grad():
    output = model.generate(
        input_ids=input_ids,
        max_length=300,  # 생성할 텍스트의 최대 길이
        temperature=1.0,  # 창의성 제어 (높을수록 랜덤한 결과)
        top_k=200,  # 높은 확률의 k개 토큰만 고려
        top_p=0.95,  # 확률의 누적 합이 0.9 이하인 토큰만 고려
        repetition_penalty=1.2,  # 반복을 줄이기 위한 페널티
        do_sample=True,  # 샘플링 사용
        num_return_sequences=3 # 3개 문장 생성하기기
)

# 생성된 텍스트 디코딩
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

# 결과 출력
print("Generated Text:")
print(generated_text[:-1])
len(generated_text)
print(type(generated_text))

Generated Text:
퇴근 후 나른한 오후에 느끼는 상반된 나의 감정, 쓸쓸함과 안락함 사이에서
혼자 걸어볼 시간조차 아까워 나는 네게 기대도 돼요
If you want to be?
You don’t never get all. but you said, no newness.
(하루종일 난 네 생각에 잠겨)
매일 밤마다 떠올리는 너와 같은 하루를 보내고 싶어
보고 싶단 말은 듣지 않아도 되는 계절이야
Can I want talk about my vibe 
Girl 모든 게 지나고 보니 또
살아가긴 힘든 거 아닐까? 하고 내게는 걱정도 됐어서
그럴 수가 있다면 네가 있어줘 제발 날 그만이라 둬주었으면 해요 이제 와서
사랑이 필요한 걸 알잖아 넌 오늘도
마음이 변했다고 해도 이 밤에 계속 외롭고 멍하니 있을 수밖에 없어
Has tell find a party with home and step that's everyday come twice i clear? When other lost form
All spending in the chastels at microw it brokeed on your organize movie activile memberges, Kickin etcurdens of mouth this jean-Minste
<class 'str'>


### 5) 텍스트 모델 개발 VER-05

In [3]:
from transformers import GPT2LMHeadModel, AutoTokenizer
import torch

# 1. 저장된 모델 및 토크나이저 로드
model = GPT2LMHeadModel.from_pretrained("./fine_tuned_new_model")
tokenizer = AutoTokenizer.from_pretrained("./fine_tuned_new_model")

model.to("cuda")
model.eval()

# 2. 프롬프트 정의
prompt = "퇴근 후 나른한 오후에 느끼는 상반된 나의 감정, 쓸쓸함과 안락함"

# 3. 텍스트 생성
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")

with torch.no_grad():
    output = model.generate(
        input_ids=input_ids,
        max_length=300,  # 생성할 텍스트의 최대 길이
        temperature=1.0,  # 창의성 제어 (높을수록 랜덤한 결과)
        top_k=200,  # 높은 확률의 k개 토큰만 고려
        top_p=0.95,  # 확률의 누적 합이 0.9 이하인 토큰만 고려
        repetition_penalty=1.2,  # 반복을 줄이기 위한 페널티
        do_sample=True,  # 샘플링 사용
        num_return_sequences=3 # 3개 문장 생성하기기
    )

# 4. 생성된 텍스트 디코딩 및 출력
generated_texts = [tokenizer.decode(out, skip_special_tokens=True) for out in output]

print("\nGenerated Texts:")
for idx, text in enumerate(generated_texts):
    print(f"{idx+1}. {text}\n")



Generated Texts:
1. 퇴근 후 나른한 오후에 느끼는 상반된 나의 감정, 쓸쓸함과 안락함
참 많이도 아파했을 거야 우리 둘의 시간 속 저편에서 보는 순간
나의 웃음이 이끄는 외로움 속에서
가끔 꿈속에서 보게 되는 이 기분은 나에게 어서 와 안녕을 찾는 날까
언제부터인 걸 어쩌면 좋아하게 된 건지 궁금해
나지막히 펼쳐 보이는 너의 눈빛도
나는 아마도 그대이길 바래
너 없이는 참 힘든 하루인데
난 널 보내야만만 해
이건 할 수 있어 난 아무말 없이 나랑 지내요
따뜻했던 그 시절의 내 모습, 마음껏 뛰어 놀던 우리 이렇게 돌아가지 마요
오랜 시간 그리웠던 우리의 마음 모두를 두고 간다면
그렇기에 나는 아직까지도 슬퍼하고 아픔에
느껴질 것 같아 그대를 봐줘요?
마음껏 뛰놀던 우리 같이 간다면 내일은 그저 힘들 뿐인가 봐요
두려워지지 않아 다시는 못해줄 테니 나와 같은 시간을 살아 준다면
온전히 함께 누리면서 살아가요 잘 달래주었으면 해 가끔씩 꿈을 찾지 않아서
내겐 언젠가의 때가 서러워서 그래 우리 둘은 시간이 멈춘 듯 하지만 매일 이별하네
고집 피우지 못했던 더 많은 아쉬움을 품고서도 
세상이 무너지던 어느 순간이 있었는지 아님 아픈 날을 견디며 서로를 위한 사랑을 가둬두었던 그때와
사랑이 주는 느낌은 잠시였기에 지금 내게 돌아와 숨 쉬어가는 모습

2. 퇴근 후 나른한 오후에 느끼는 상반된 나의 감정, 쓸쓸함과 안락함
오래 전부터 너에게 닿았던 눈처럼 
모든 게 낯설지만 그 때마다 난 다시 그렇게 살고 싶어 
살아 갈 수 있다면 바랄게 없는 걸 알기에 안 그래요
참 고민이 많던 밤 너와 나를 비교하려 해봐도
그럼에도 다 내겐 없는데
I don't know you were forever into the sunlight 
꿈과 상상 속 환상 속으로 사라져간 나는 사라져가겠지 
내가 있든 없든, 많은 시간 속에 있어줘 여전히 너의 모습이 느껴져 여기 머무르고 싶은 날
이대로 잠들자 널 잊혀두고 그냥 걸어갈 거야 잠시만 쉬고 싶을 뿐인데.
다시 한 번 약속

In [6]:
from evaluate import load
import numpy as np

# 1. 평가 지표 불러오기
bleu = load("bleu")
rouge = load("rouge")

# 2. 테스트 데이터에서 일부 샘플 가져오기
num_samples = 10
test_samples = val_texts.sample(num_samples).tolist()

# 3. 모델 예측 및 평가
references = []
predictions = []

for sample in test_samples:
    input_ids = tokenizer(sample, return_tensors="pt").input_ids.to("cuda")
    
    with torch.no_grad():
        output = model.generate(input_ids=input_ids, max_length=100)
    
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    
    references.append([sample])  # BLEU 평가는 다중 참조 가능하므로 리스트로 감싸기
    predictions.append(generated_text)

# 4. BLEU 점수 계산
bleu_score = bleu.compute(predictions=predictions, references=references)
print(f"BLEU Score: {bleu_score['bleu']:.4f}")

# 5. ROUGE 점수 계산
rouge_score = rouge.compute(predictions=predictions, references=references)
print(f"ROUGE Score: {rouge_score}")

# 6. Perplexity(혼란도) 계산
def compute_perplexity(model, text):
    inputs = tokenizer(text, return_tensors="pt").to("cuda")
    with torch.no_grad():
        outputs = model(**inputs)
    loss = outputs.loss.item()
    return np.exp(loss)

perplexities = [compute_perplexity(model, text) for text in test_samples]
avg_perplexity = np.mean(perplexities)
print(f"Perplexity: {avg_perplexity:.4f}")


NameError: name 'val_texts' is not defined

### 6) 텍스트 모델 원본 (VER-04,05)

In [4]:
from transformers import GPT2LMHeadModel, AutoTokenizer

# 저장된 모델 및 토크나이저 로드
model = GPT2LMHeadModel.from_pretrained("skt/kogpt2-base-v2")
tokenizer = AutoTokenizer.from_pretrained("skt/kogpt2-base-v2")

# 모델을 GPU로 이동
model.to("cuda")
model.eval()  # 평가 모드로 전환 (dropout 비활성화)

# 프롬프트 정의
prompt = "퇴근 후 나른한 오후에 느끼는 상반된 나의 감정, 쓸쓸함과 안락함"

# 텍스트를 토큰화
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")

# 텍스트 생성
output = model.generate(
    input_ids=input_ids,
    max_length=300,  # 생성할 텍스트의 최대 길이
    temperature=1.0,  # 창의성 제어 (높을수록 랜덤한 결과)
    top_k=200,  # 높은 확률의 k개 토큰만 고려
    top_p=0.95,  # 확률의 누적 합이 0.9 이하인 토큰만 고려
    repetition_penalty=1.2,  # 반복을 줄이기 위한 페널티
    do_sample=True,  # 샘플링 사용
    num_return_sequences=3 # 3개 문장 생성하기기
)

# 생성된 텍스트 디코딩
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

# 결과 출력
print("Generated Text:")
print(generated_text[:-1])
len(generated_text)
print(type(generated_text))

Generated Text:
퇴근 후 나른한 오후에 느끼는 상반된 나의 감정, 쓸쓸함과 안락함 사이에서 고민해보고자 하는 모든 이들이 바라는 당신만의 여행이다.
하지만 누군가는, 여행을 떠난다면 조금 더 천천히 즐길 시간이 필요하다.
그리고 마이크를 잡고 가 봤다.
어느새 내겐 아름다운 자연을 배경으로 펼쳐진 산책로가 펼쳐지기 시작했다.
먼저 눈에 들어온 것은 우거진 초원 위로 우뚝 솟은 전망대를 든 한 관광객. 넓은 해변으로 뻗은 돌담길을 지나 계곡을 따라 걸어 내려갔다.
나무울음 소리만 들린 숲은 시원하게 바람에 흔들리고 있었다.
원주민의 웃음소리를 제대로 듣고 싶었다.
자연 속에 있는 사람이라면 누구나 느낄 수 있을까?
시동을 걸면 이글거리는 눈발을 확인할 길이 있다.
수줍은 표정의 어른 키만한 코트에 흰 와이셔츠까지 겹쳐졌다.
아이들이 좋아하는 꽃무늬 셔츠의 모습이 눈에 들어왔고 두 손으로 손가방을 움켜쥐며 서있는 아빠의 모습도 떠올랐다.
기다림의 미학은, 눈을 뜬 순간이 온 힘을 다해 보고 싶다는 욕심으로 오는 거였다.
아이와 함께 걷는 길을 선택한 것도 쉽지 않았다.
내게 있어서 좋은 아이디어야
동양에서는 별난 존재다.
요즈음 아이가 자라면서 가장 많이 발생하는 호흡기 질환을 일컫는 '호흡기계 증후군'을 흔히 접하게 된다.
대부분의 어린 시절이 소아기의 초기 증상인데 그 전에 어느 시기부터 고령의 부모들은 아이들이 숨을 쉴 때 손을 모아 기도를 들이마시고 기침을 하고 나서 약 30분간 인공 호흡을 한다.
그만큼 호흡하기가 힘들다고 알고 있기 때문이며 뇌세포
<class 'str'>


In [3]:
tokenizer.special_tokens_map

{'bos_token': '<|endoftext|>',
 'eos_token': '<|endoftext|>',
 'unk_token': '<|endoftext|>',
 'pad_token': '<|endoftext|>'}